# **Solving the time-independent Schrödinger equation in 1D**

## Importing the libraries used

In [7]:
import numpy as np

from math import pi, sqrt, sin, factorial
from scipy.integrate import quad

In [8]:
# Debuging mode ?
debug = True

In [9]:
# Typing
type Vector = np.ndarray[float]
type Family = list[Vector]
type Matrix = list[list[float]]

## Diagonalization procedure

### Orthonormalisation procedure useful for the Davidson method
We use the gramschmidt algorithm (https://fr.wikipedia.org/wiki/Algorithme_de_Gram-Schmidt)

In [10]:
def matrix_to_family(M: Matrix) -> Family:
    """ Transform a matrix into a family of vectors constituting the columns of this matrix """
    return np.array(M).T

def family_to_matrix(L: Family) -> Matrix:
    """ Transform a family of vectors into a matrix """
    return np.array(L).T

if debug:
    M = np.random.randint(100, size=(10,10))
    print("We want to transform a matrix M : \n\n", M, "\n\ninto a family of vectors : \n")
    print(matrix_to_family(M))
    print("\nIt has been transformed back into a matrix :\n")
    print(family_to_matrix(matrix_to_family(M)))

On veut transformé un matrice M : 

 [[13 66 58 61 65 72 67  6 89  2]
 [59 50 20 66 37  9 55 71 72 40]
 [70 95 70 98 57 54 38 26 95 34]
 [83 74 56 96 64 83 58 16 53  1]
 [65 48 57 31 49 48 42 96 49 50]
 [19 36 84 82 32 62 40  9 68 42]
 [46 81 44 10  3 64 38 60 54 17]
 [ 7 77  3  4 10 54 22 51 68 18]
 [66 44  7 55 23 23 41 39 81 28]
 [48 86 32 33 48 11 37 77  1 78]] 

en famille de vecteur : 

[[13 59 70 83 65 19 46  7 66 48]
 [66 50 95 74 48 36 81 77 44 86]
 [58 20 70 56 57 84 44  3  7 32]
 [61 66 98 96 31 82 10  4 55 33]
 [65 37 57 64 49 32  3 10 23 48]
 [72  9 54 83 48 62 64 54 23 11]
 [67 55 38 58 42 40 38 22 41 37]
 [ 6 71 26 16 96  9 60 51 39 77]
 [89 72 95 53 49 68 54 68 81  1]
 [ 2 40 34  1 50 42 17 18 28 78]]

On l'a retransforme en matrice :

[[13 66 58 61 65 72 67  6 89  2]
 [59 50 20 66 37  9 55 71 72 40]
 [70 95 70 98 57 54 38 26 95 34]
 [83 74 56 96 64 83 58 16 53  1]
 [65 48 57 31 49 48 42 96 49 50]
 [19 36 84 82 32 62 40  9 68 42]
 [46 81 44 10  3 64 38 60 54 17]
 [ 7 77

In [11]:
def ps(a: Vector, b: Vector) -> float:
    """
    Make the canonical scalar product of ℝ^n explain in
    https://fr.wikipedia.org/wiki/Produit_scalaire_canonique
    """
    sum = 0
    for i in range(len(a)):
        sum += a[i]*b[i]
    return sum

if debug:
    u = [1,2,3]
    v = [2,3,4]
    print(f"The canonical scalar product of {u} with {v} which should give 20")
    print("résultat = ", ps(u, v))

On fait le produit scalaire canonique de [1, 2, 3] avec [2, 3, 4] qui doit donner 20
résultat =  20


In [12]:
def norm(u: Vector) -> float:
    """ Norm associated with the canonical scalar product of ℝ^n """
    return ps(u, u)**(1/2)

if debug:
    u = np.array([1, 1, 1])
    print(f"The norm of {u} associated with the canonical saclary product of ℝ^n which must give √3")
    print("result = ", norm(u))

On calcule la norme de [1 1 1] associé au produit saclaire canonique de ℝ^n qui doit donner √3
résultat =  1.7320508075688772


In [13]:
def orthonorm_vec(F: Family, v: Vector) -> Vector:
    """ Orthonomalises a vector with respect to a family of orthonormal vectors """
    u = np.array(v)
    for vk in np.array(F):
        u = u - ps(vk, v)*vk
    return u/norm(u)

if debug:
    F = [[1, 0, 0],[0, 1, 0]]
    v = [1, 1, 1]
    print(f"Orthonormalise the vector {v} with respect to the orthonormal family \n{F}\n must return the vector (0,0,1)")
    print("result = ", orthonorm_vec(F, v))    

Orthonormalise le vecteur [1, 1, 1] par rapport à la famille orthonormale 
[[1, 0, 0], [0, 1, 0]]
 doit renvoyer le vecteur (0,0,1)
résultat =  [0. 0. 1.]


In [14]:
def gram_schmidt(F: Family) -> Family:
    """ Apply the gramschmidt algorithm to orthonomise any family F """
    ONF = []
    for e in F:
        ONF.append(orthonorm_vec(ONF, e))
    return np.array(ONF)

if debug:
    F = [[1, 0, 0],[1, 1, 0],[1, 1, 1]]
    print("The family F is orthonormalized: ")
    print(F)
    print("We need to find [[1, 0, 0],[0, 1, 0],[0, 0, 1]]")
    print("result : ", gram_schmidt(F))

On orthonormalise la famille F : 
[[1, 0, 0], [1, 1, 0], [1, 1, 1]]
On doit trouver [[1, 0, 0],[0, 1, 0],[0, 0, 1]]
résultat :  [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [15]:
def gram_schmidt_matrix(M: Matrix) -> Matrix:
    """ Apply the gramschmidt algorithm to orthonomalise the matrix M. """
    return family_to_matrix(gram_schmidt(matrix_to_family(M)))

if debug:
    M = np.random.randint(100,size=(3, 3))
    print("We want to orthonormalise M :")
    print(M)
    print("result :")
    ONM = gram_schmidt_matrix(M)
    print(ONM)
    print("Is the matrix orthonormal?")
    print("norm : ", norm(ONM[:,0].T))
    print("norm : ", norm(ONM[:,1].T))
    print("norm : ", norm(ONM[:,2].T))
    print("dot product : ", ps(ONM[:,0].T, ONM[:,2].T,))
    print("dot product : ", ps(ONM[:,0].T, ONM[:,1].T,))
    print("dot product : ", ps(ONM[:,2].T, ONM[:,1].T,))

On veut orthonormaliser M :
[[72 50 80]
 [28 22  3]
 [26 15 97]]
résultat :
[[ 0.88331923  0.0214682  -0.4682801 ]
 [ 0.34351303  0.6500934   0.67777384]
 [ 0.31897639 -0.75955098  0.56686539]]
La matrice est orthonormale ?
norme :  1.0
norme :  1.0
norme :  0.9999999999999999
produit scalaire :  5.467848396278896e-15
produit scalaire :  1.4432899320127035e-15
produit scalaire :  -6.6058269965196814e-15


### Procédure de tri pour la méthode de Davidson
On utilise la méthode de trifusion qui est en complexité n*log(n) (https://fr.wikipedia.org/wiki/Tri_fusion)

In [16]:
def fusion(A: list, B: list) -> list:
    """ Procedure that merges two sorted lists """
    if len(A) == 0:
        return B
    elif len(B) == 0:
        return A
    elif A[0] <= B[0]:
        return [A[0]] + fusion(A[1:], B)
    else:
        return [B[0]] + fusion(A, B[1:])
if debug:
    A = [1, 3, 4, 7]
    B = [2, 3, 5]
    print(f"We merge the lists {A} and {B} to obtain this list: [1, 2, 3, 3, 4, 5, 7].")
    print("result : ", fusion(A, B))

On fusione les listes [1, 3, 4, 7] et [2, 3, 5] pour obtenir cette liste : [1, 2, 3, 3, 4, 5, 7]
résultat :  [1, 2, 3, 3, 4, 5, 7]


In [17]:
def triFusion(L):
    """ Sort a list using the divide and conquer principle """
    if len(L) == 1:
        return L
    else:
        return fusion(triFusion(L[:len(L)//2]) , triFusion(L[len(L)//2:]))

if debug:
    L = list(zip([1, 76, 71, 6, 25, 50, 20, 18, 84, 11],[1,1,1,1,1,1,1,1,1,1]))
    print("We sort the list:", L, "according to the first value of the pair")
    print("result :", triFusion(L))

On tri la liste : [(1, 1), (76, 1), (71, 1), (6, 1), (25, 1), (50, 1), (20, 1), (18, 1), (84, 1), (11, 1)] selon les premières valeur du couple
résultat : [(1, 1), (6, 1), (11, 1), (18, 1), (20, 1), (25, 1), (50, 1), (71, 1), (76, 1), (84, 1)]


### Méthode de Davidson
Implémentation basé sur ces explications page 65 : https://www.irisa.fr/sage/bernard/publis/DAVIDSON94.pdf

In [18]:
def davidson(M: Matrix, m=3, l=1, seuil=1e-8, MIt=600) -> tuple:
    """
    M: the matrix to be diagonalized
    m: the precision of the agorithm Km = {V0;MV0;M²V0;...}
    l: number of eigenvectors/values to find
    seuil: precision of the eigenvalue aproxiamtion
    MIt : maximum of itieration

    Function that returns the cut ([eigenvalue], [eigenvector]) of the matrix M
    """
    np.set_printoptions(precision=3, suppress=True)

    # Initialisation
    M = np.array(M)
    N = len(M)
    V1 = np.eye(N, l)
    I = np.identity(N)

    if not(l<=m<=N):
        raise ValueError(f"value must be {l} <= {m} <= {N}")

    # Construction of the reduced subspace (of Kernal)
    v = [V1]

    # iteration
    for k in range(MIt):
        # Solving the reduced subspace
        T = np.dot(v[k].T, np.dot(M, v[k]))
        val, vec = np.linalg.eig(T)
        #val, vec = lanczos(T)
        Eigencouple = list(zip(val,vec))
        Eigencouple = triFusion(Eigencouple)[-l:] # Plus grande VP
        #Eigencouple = triFusion(Eigencouple)[:l] # Plus petite VP
        val, vec = list(zip(*Eigencouple))
        vec = np.array(vec).T
        val = np.array(val)

        x = np.array([[] for _ in range(N)])
        r = np.array([[] for _ in range(N)])
        t = np.array([[] for _ in range(N)])

        # Subspace expansion
        for i in range(l):
            yi = vec[:, i].reshape((T.shape[0], 1))

            xi = np.dot(v[k], yi)
            x = np.concatenate((x, xi), axis=1)

            ri = val[i]*xi-np.dot(M, np.dot(v[k], yi))
            r = np.concatenate((r, ri), axis=1)

            # Threshold at 1e-300 to avoid problems of division by 0
            mu_i = [1/max(np.abs(val[i]-M[j, j]), 1e-200)
                    for j in range(N)]
            Ci = np.diag(mu_i)
            ti = np.dot(Ci, ri)
            t = np.concatenate((t, ti), axis=1)
                
        # Stop if the algo converges 
        if np.linalg.norm(t) < seuil:
            print("Davidson converges")
            return val, x

        # Otherwise, a new pass matrix is regenerated
        if v[k].shape[0] <= m - l :
            v.append(gram_schmidt_matrix(np.concatenate((v[k], t), axis=1)).reshape(N, v[k].shape[1]+l))
        else:
            v.append(gram_schmidt_matrix(np.concatenate((x, t), axis=1)).reshape(N, 2*l))

    return val, x

if debug:
    M = [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]]
    print("We are looking for the eigenvalues and eigenvectors of : \n", M)
    print("result : ", davidson(M))
    print("theoretical result :", np.linalg.eig(M))

On cherche les valeurs et vecteurs propres de : 
 [[1, 1, 1], [1, 1, 1], [1, 1, 1]]
résultat :  (array([1.]), array([[1.],
       [0.],
       [0.]]))
résultat théorique : EigResult(eigenvalues=array([ 3., -0., -0.]), eigenvectors=array([[ 0.577, -0.   , -0.811],
       [ 0.577, -0.707,  0.325],
       [ 0.577,  0.707,  0.486]]))


C:\Users\rodri\AppData\Local\Temp\ipykernel_9904\3504234912.py:8: RuntimeWarning: overflow encountered in scalar multiply
  sum += a[i]*b[i]


### Lanczos method
Implementation based on these explanations https://en.wikipedia.org/wiki/Lanczos_algorithm

In [19]:
# In construction
def lanczos():
    pass

## Construction of the basis for solving the equation

### Base of sinuses
> ⚠️⚠️**Please note that the calculations made by hand are undoubtedly incorrect as they differ from the numerical version.** ⚠️⚠️

This function is used to initialise the B_sin class, in particular by initialising the values of the various variables.
This class has the following methods:

- init :
This function initialises the various variables associated with our B_sin class.

- ret_base :
This function returns the base composed of sines of the desired size.

- prdt_scalaire_Ec :
Returns the hand-calculated dot product of the kinetic energy of the Hamiltonian.

- prdt_scalaire_Ep :
Returns the hand-calculated dot product of the potential energy of the Hamiltonian.

In [20]:
class B_sin():
    """ This class can be used to calculate the sine base for the Hamiltonian """
    def __init__(self, N, L, V):
        """ Method for initializing the B_sin() object class """
        self.N = N
        self.L = L
        self.V = V
        x = sqrt(2)
        self.k = eval(self.V)
    
    def ret_base(self):
        """ Method for calculating the list of sines that will form the base """
        return [f'sqrt(2/{self.L}) * sin(({n}*pi*x)/{self.L})' for n in 
                range(self.N)]

    def prdt_scalaire_Ec(self, i, j):
        """ Method giving the scalar product: <Φi|T̂|Φj> (manual calculation) """
        if i == j:
            return - (j*pi)**2 / self.L
        else :
            return 0
    
    def prdt_scalaire_Ep(self, i, j):
        """ Method giving the scalar product: <Φi|V̂|Φj> (calculated by hand) for a harmonic potential """
        if i == j :
            return (self.k * (self.L)**2) / 3
        else :
            return ((2 * self.k * (self.L)**2) / pi**2) * ( (((-1)**(i+j))/(i+j)**2) - (((-1)**(i-j))/(i-j)**2) )

if debug:
    N = 3 # The dimenssion of the base
    L = 1 # in A.U. is the size of the infinite well where the sines are eigenvectors of the Hamiltonian
    m = 1 #  in A.U. is the mass of the particle
    w = 1 # in A.U. is the pulsation of the vibration
    h = 1 # in A.U. is Plank's constant
    k = m*(w**2) # spring stiffness
    V = f"({k}*x**2)/2" # the potential of an infinite well
    B = B_sin(N, L, V)
    print(f"The sine basis for the infinite well of length {L} A.U. consisting of {N} vectors is :")
    print(B.ret_base())
    i = 0
    j = 0
    print(f"The value of the kinetic energy of the system for the vectors {i+1} and {j+1} is : ", B.prdt_scalaire_Ec(i,j))
    print(f"The value of the potential energy of the system for the vectors {i+1} and {j+1} is : ", B.prdt_scalaire_Ep(i,j))

La base des sinus qui est base pour le puit infini de longueur 1 U.A. constituer de 3 vecteurs est :
['sqrt(2/1) * sin((0*pi*x)/1)', 'sqrt(2/1) * sin((1*pi*x)/1)', 'sqrt(2/1) * sin((2*pi*x)/1)']
La valeur de l'énergie cinétique du système pour les vecteurs 1 et 1 est :  -0.0
La valeur de l'énergie potentiel du système pour les vecteurs 1 et 1 est :  0.3333333333333334


### Other basis

We have also drafted a basis with a harmonic oscillator, which you will find commented on at the end of the Python document 'base'.  
**The different sources:**  
* https://fr.wikipedia.org/wiki/Oscillateur_harmonique_quantique  
* https://fr.wikipedia.org/wiki/Polyn%C3%B4me_d%27Hermite



### Any base
Here we want to create a base class which will create a base in comprehension. 

Notre class base aurra besoin de calculer la dériver seconde d'une fonction en un point

In [21]:
def deriv(f : str, x : float, h = 1e-3):
    """ 
    Fonction qui calcule la dériver de la fonction f en x.
    f est un string.
    Exemple d'utilisation pour deriver la fonction sinus en 0:
    f = 'np.sin(x)'
    x = 0.
    resultat = deriv(f,x)
    """
    x += h
    res = eval(f)
    x -= 2 * h
    res -= eval(f)
    return res/(2*h)

def derivderiv(func : str, x : float, h = 1e-3):
    """
    Fonction qui calcule la dériver seconde de la fonction f en x.
    f est un string.
    Exemple d'utilisation pour deriver deux fois la fonction sinus en 0:
    f = 'np.sin(x)'
    x = 0.
    resultat = derivderiv(f,x)
    """
    return (deriv(func,x+h)-deriv(func,x-h))/(2*h)

if debug:
    f = "np.sin(x)"
    x = 0.
    print(f"La dériver de {f} en {x} est ", deriv(f,x))
    print(f"La dériver seconde de {f} en {x} est ", derivderiv(f,x))

La dériver de np.sin(x) en 0.0 est  0.9999998333333416
La dériver seconde de np.sin(x) en 0.0 est  0.0


In [22]:
class base():
    """ Cette classe permet de calculer une base quelconque pour le calcul de l'hamiltonien """
    def __init__(self, N : int, func : str, var : str, pot : str, L : float):
        """ 
        Methode qui initialise la classe de l'objet base.
        N : Le nombre de vecteur de la base
        func : est un string de la fonction comme si on l'écrivait. ex: func = "np.sin(n*x)"
        var : est la variable d'itération de la base. ex: avec l'exemple précédant on itère selon la variable n : (sin(x), sinc(2x), sin(3x),...)
        pot : le potentiel associé au problème défini selon les même restriction que func
        L : La largeur de la boîte de potentiel qui comprend le potentiel définit
        """
        self.V = pot
        # create the base
        self.base =[]
        self.L = L

        function = func.split(var)
        for n in range(1,N+1):
            f = ''
            for i in range(len(function)):
                if i != len(function) - 1:
                    f += function[i] + str(n)
                else:
                    f += function[i]
            self.base.append(f)

    def prdt_scalaire_Ec(self, i, j) -> float:
        """ Methode qui donne le produit scalaire : <Φi|T̂|Φj> """
        def func(x):
            x = x
            return eval(self.base[i]) * derivderiv(self.base[j],x)
        return quad(func, -self.L, self.L)[0]
    
    def prdt_scalaire_Ep(self, i, j) -> float:
        """ Methode qui donne le produit scalaire : <Φi|V̂|Φj>  pour un potentiel quelconque"""
        def func(x):
            x = x
            return eval(self.base[i]+'*'+self.V+'*'+self.base[j])
        return quad(func, -self.L, self.L)[0]

if debug:
    N = 3 # La dimenssion de la base
    L = 1 # en U.A. est la taille du puit infini où les sinus sont vecteurs propres de l'hamiltonien
    m = 1 # en U.A. est la masse de la particule
    w = 1 # en U.A. est la pulsation de la vibration
    h = 1 # en U.A. est la constante de Plank
    k = m*(w**2) # la raideur du ressort
    V = f"({k}*x**2)/2" # le potentiel d'un puit infini

    f = "np.sin(a*x)"
    var = "a"
    
    # Définition de la base
    B = base(N, f, var, V, L)
    print(f"La base des {f} qui est contenue dans un puit infini de longueur {L} U.A. constituer de {N} vecteurs est :")
    print(B.base)
    i = 0
    j = 0
    print(f"La valeur de l'énergie cinétique du système pour les vecteurs {i+1} et {j+1} est : ", B.prdt_scalaire_Ec(i,j))
    print(f"La valeur de l'énergie potentiel du système pour les vecteurs {i+1} et {j+1} est : ", B.prdt_scalaire_Ep(i,j))

La base des np.sin(a*x) qui est contenue dans un puit infini de longueur 1 U.A. constituer de 3 vecteurs est :
['np.sin(1*x)', 'np.sin(2*x)', 'np.sin(3*x)']
La valeur de l'énergie cinétique du système pour les vecteurs 1 et 1 est :  -0.5453511048073206
La valeur de l'énergie potentiel du système pour les vecteurs 1 et 1 est :  0.15704119745024206


## Resolution routine
### Procedure which determines the Hamiltonian of the system associated with the basis of description

In [23]:
def CalculHamiltonian(base, N):

    H = np.zeros((N,N))

    for i in range(N):
        for j in range(i + 1):
            H[i,j] = base.prdt_scalaire_Ec(i, j) +  base.prdt_scalaire_Ep(i, j)

    H = H + H.T - np.diag(np.diag(H))

    return H

In [24]:
### Constants and functions
N = 10 # The dimenssion of the base
L = 1 # in A.U. is the size of the infinite well where the sines are eigenvectors of the Hamiltonian
m = 1 # in A.U. is the mass of the particle
w = 1 # in A.U. is the pulsation of the vibration
h = 1 # in A.U. is Plank's constant
k = m*(w**2) # spring stiffness
V = f"({k}*x**2)/2" # the potential of an infinite well

func = f'np.sqrt(2/{L})*np.sin( (a*np.pi*x) /{L})'
var = "a"

In [26]:
B = base(N, func, var, V, L)
print("The basis of the system is :")
print(B.base)

H = CalculHamiltonian(B, N)
print()
print("The associated Hamiltonian is :")
print(H)

print()
print("System resolution in progress ...")
E, phi = davidson(H)
val, vec = np.linalg.eig(H)

print()
print("Resolution complete.")
print("Digital energy : ", E)
print("Digital energy with numpy : ", val)
print("Analitic energy : ", h*w/2)

The basis of the system is :
['np.sqrt(2/1)*np.sin( (1*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (2*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (3*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (4*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (5*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (6*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (7*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (8*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (9*np.pi*x) /1)', 'np.sqrt(2/1)*np.sin( (10*np.pi*x) /1)']

The associated Hamiltonian is :
[[ -19.456   -0.18     0.038   -0.014    0.007]
 [  -0.18   -78.635   -0.195    0.045   -0.018]
 [   0.038   -0.195 -177.32    -0.199    0.047]
 [  -0.014    0.045   -0.199 -315.481   -0.2  ]
 [   0.007   -0.018    0.047   -0.2   -493.108]]

System resolution in progress ...

Resolution complete.
Digital energy :  [-19.456]
Digital energy with numpy :  [ -19.456 -493.109  -78.635 -177.32  -315.481]
Analitic energy :  0.5
